<div style='width=100%; display:flex;flex-direction:row'><img  src=https://universidadeuropea.com/resources/media/images/universidad-europea-logo_poc9mEM.original.png width=100  style='  margin-left: auto;margin-right: auto; width: 25%; height:25%;'><img  src=https://i.ibb.co/1068C7j/EATEASER.jpg width=100 style='  margin-left: auto;margin-right: auto; width: 10%;height:25%;'></div>

<div style='margin:auto;text-align: center;font-family: "Times New Roman", Times, serif; font-weight: bold;'>PROYECTO COMUTACIONAL<br><br>EATEASER - VOZ A TEXTO</div>

<div style='width:100%; display:flex;flex-direction:row'>
    <div style='width:50%;margin-right:5cm;'>
        <p style='font-family: "Times New Roman", Times, serif; font-weight: bold;'>ESTUDIANTES</p>
<ul style='font-family: "Times New Roman", Times, serif;'>
    <li>Adilem Dobras 21911633</li><li>Roberto Echevarria 21823680</li><li>Carlos Gonzales 22067726</li><li>Juan Carlos Rondeau 21816176</li></ul> </div>

</div>

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">1. Importamos las librerias</h2>

In [1]:

!pip install pytube
!pip install SpeechRecognition
!pip install pydub
!pip install moviepy
!pip install beautifulsoup4
!pip install nltk



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pytube import YouTube
import speech_recognition as sr
from pydub import AudioSegment
import moviepy.editor as mp
from bs4 import BeautifulSoup
import glob
import os
import re
from os import listdir
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import requests
from pydub.silence import split_on_silence

C:\Users\pitu\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">2. Inicio del programa</h2>

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE CONTROLADORVIDEO</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará los ajustes para manejar el video recibido y manipularlo.</p>


In [125]:
class ControladorVideo:
    def __init__(self,enlace): 
        self._idvideo = 0
        self.enlacevideo=enlace
        self.yt=YouTube(self.enlacevideo)
        self.nombrevideo=''
        self.titulovideo=self.yt.title
        self.autorvideo=self.yt.author
        self.fechavideo=self.yt.publish_date
        self.duracionvideo=self.yt.length
    """|DESCARGAR VIDEO URL: descarga el video de youtube
       |return: devuelve una ruta absoluta"""
    def descargarVideoURL(self):
        recetasVideos = 'recetasvideos/'
        #aqui creo un nuevo id para el nuevo video
        self._idvideo= self._idvideo+1
        #esta sera el archivo del video y su nuevo nombre
        nombre='receta'+str(self._idvideo)
        #le pedimos al pytube que solo nos descargue el audio y lo descargamos
        t=self.yt.streams.filter(file_extension='mp4').first().download(output_path=recetasVideos,filename=nombre+'.mp4')
        #devolvemos el nombre
        return nombre
    """|PARSEO VIDEO: pasa el video de .mp4 a .wav
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve el nuevo nombre del audio en .wav"""
    def parseoVideo(self,nombre):
        recetasVideos = 'recetasvideos/'
        #tomamos el video en mp4 
        track = mp.VideoFileClip(recetasVideos+nombre+'.mp4')
        #cambiamos el video a .wav
        nombre_wav="{}.wav".format(nombre)
        track.audio.write_audiofile(recetasVideos+nombre_wav)
        track.close()
        return nombre
   # def reenumerar(): """ EN PROCESO, para leer el directorio y añadir receta n + 1 """
       # dir = []
        #for archivo in listdir("./recetasvideos"):
        #    dir.append(archivo)
        #print(dir)
    """|SPEECH TEXT:Transforma el audio a texto
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve un string con el texto devuelto"""
    def speech_text(self,nombre):
        recetasVideos = 'recetasvideos/'
        #instanciamos el recognizer
        print(recetasVideos+nombre)
        r = sr.Recognizer()
        audio = sr.AudioFile(recetasVideos+nombre)
        with audio as source:
            audio_file = r.record(source)
        #transcribimos el audio a texto
        result = r.recognize_google(audio_file, language = 'es-ES')
        return result
            
        
        

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE DEPURADOR</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará el proceso de extraccion, transformacion y carga de nuestro programa EATEASER.</p>


In [126]:
#aqui iran lecturas-escrituras-guardar-eliminar cosas en bases de datos
class RecursosAdicionales:
    """|ESCRITURA: escribe textos txt
       |nombre: nombre del 
       |return: devuelve el audio en texto"""
    #yo siento que en vez de cambiar de carpeta los textos...deberiamos eliminar los mp4
    #y si lo mas factible aqui es que guardemos el video por su nombre?
    #creo que no debemos contar los files porque cansan el programa
    
    def escritura(self,nombre,texto):
        recetasTextos = 'recetastextos/'
        if not(os.path.exists(recetasTextos)):
            os.mkdir(recetasTextos)
        f = open(recetasTextos+nombre+'.txt', 'w')
        f.write(texto)
        f = open(recetasTextos+nombre+'.txt', "r")
        print(f.read())
        f.close()
    def eliminacionMp4(self):
        recetasVideos = './recetasvideos/'
        py_files = glob.glob(recetasVideos+'*.mp4')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}")      
        

In [127]:
#si el video es mayor de 3 minutos no funciona
#si el video es en ingles no funciona
class Depurador:
    def __init__(self): 
        self.rec=RecursosAdicionales()
    """|EXTRACCION: proceso etl donde extraemos al informacion del video 
       |enlace: es un string que se colocara el enlace del video"""
    def extraccion(self,enlace):
        #instanciamos el controlador de videos
        cv=ControladorVideo(enlace)
        #paso 1: descargamos el video
        cv.nombrevideo=cv.descargarVideoURL()
        #paso 2: pasamos el video a .wav
        nombre=cv.parseoVideo(cv.nombrevideo)
        #paso 3: pasamos el .wav a texto
        num_segm=segcionarXsilencios(nombre)
        result=""
        print("hola")
        for i in range(num_segm):
            try:
                
                result=result+str(cv.speech_text("../temp/{}_extracto{}.wav".format(nombre,i+1)))
                result=result+" "
            except:   
                
                audio1=aseg.from_wav("temp/{}_extracto{}.wav".format(nombre,i+1))
                duracion=audio1.duration_seconds
                if duracion<=5:
                    print("El extracto {} es un silencio".format(i+1))
                elif duracion<=180:
                    print("El extracto {} es música o ruido".format(i+1))
                else:
                    print("Error importante en el extracto {}".format(i+1))
                
            
                
        
        #paso 4: escribimos el texto recibido en un txt
        #print("Titulo:"+cv.titulovideo+"\n"+"Autor:"+cv.autorvideo+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
        resultado=self.rec.escritura(cv.nombrevideo,"Titulo:"+cv.titulovideo+"\n"+"Autor:"+cv.autorvideo+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
        #paso 5: eliminamos los mp4
        self.rec.eliminacionMp4()
        return resultado     
    def transformacion(self):
        print()
    def carga(self):
        print()

In [128]:
import numpy as np
from pydub import AudioSegment as aseg
def segcionarXsilencios(audio):
    
    audio1=aseg.from_wav("./recetasvideos/"+audio+".wav")
    
    var_min=1900
    salir=False
    while salir==False:
        
        # this is an array
        samples = audio1.get_array_of_samples()

        
        segundo=88521
        index=[]
        for i in range(0,len(samples),int(segundo/5)):
            dataSeg = samples[i:int(segundo/5)+i]
            media=np.mean(dataSeg)
            var=np.var(dataSeg)
            if -10<=media<=10 and var<=var_min:
                index.append(i)
        '''
        for i in range(len(index)):
            print(int(index[i])/88521)
        '''
        borrar=[]


        guardado=0
        for i in range(len(index)-1):


            if index[i+1]<=index[i]+(20*segundo):

                if i==0:
                    tiempo=(index[i])/segundo
                else:
                    tiempo=(index[i+1]-guardado)/segundo

                #print("tiempo: "+ str(tiempo))

                if tiempo<=120:
                    borrar.append(i)
                else:
                    #print("guardados: " + str(index[i]/segundo))
                    guardado=index[i]

            else:
                
                guardado=index[i]


    
        
        final=np.delete(index, borrar, axis=0) 
        #print("------------------------------------------------")
        #print(borrar)
        #print("------------------------------------------------")
        #print(final)
        #print("length final: " + str(len(final)))
        extractos=[]
        if len(final)==0:
            #print("if")
            var_min=var_min*10
            salir=False
        else:
            #print("else")
            for i in range(len(final)):
                if i==0:

                    extractos.append(samples[:final[i]])
                else:
                    extractos.append(samples[final[i-1]:final[i]])
            #print(len(extractos))
            extractos.append(samples[final[i]:])
            
            #print(len(extractos))
            salir=True
            
    for i in range(len(extractos)):
        nombre=""
        new_sound = audio1._spawn(extractos[i])
        nombre="temp/{}_extracto{}.wav".format(audio,i+1)
        new_sound.export(nombre,format="wav")
    print(len(extractos))
    return len(extractos)
    

In [129]:
class WebScrap:
    def __init__(self): 
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    def request(self, url):
        request1 = requests.get(url, headers=self.headers)
        html = request1.content
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    def verificar_alimento(self,alimento):
        soup = self.request( 'https://www.themealdb.com/api/json/v1/1/search.php?s='+alimento)
        print(soup)
        

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">3. Main</h2>

In [124]:
#y si tambien vemos si le permitimos al usuario que meta videos?
dep=Depurador()
if __name__ == '__main__':
    dep.extraccion('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')
    dep.extraccion('https://www.youtube.com/watch?v=PsqR5M8rdjA&list=LL&index=9&t=4s')
    dep.extraccion('https://www.youtube.com/watch?v=xfYcM_jHgPY')
    dep.extraccion('https://www.youtube.com/watch?v=DnamuF14sJ0')
    dep.extraccion('https://www.youtube.com/watch?v=JjXFsmoiLl8')
    dep.extraccion('https://www.youtube.com/watch?v=ZaCIElVBI9Y&list=PLNqFB7lW8Xrowk5ehMiwURtUvrGpgqSYW&index=2')
    dep.extraccion('https://www.youtube.com/watch?v=XsdSJSSLB88&list=PLNqFB7lW8Xrowk5ehMiwURtUvrGpgqSYW&index=25')
    dep.extraccion('https://www.youtube.com/watch?v=NoubmBJosqY')
   
    
    

MoviePy - Writing audio in recetasvideos/receta1.wav


MoviePy - Done.
4
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
El extracto 4 es un silencio
Titulo:Albóndigas de Merluza en Salsa Verde | Recetas de PESCADO Faciles
Autor:Platos Fáciles con Tamara
Fecha Publicacion:2013-07-04 00:00:00
Entradilla:hoy vamos a hacer albóndigas de merluza en salsa verde y los ingredientes que vamos a utilizar son 500 gramos de merluza limpios de piel y espinas 3 dientes de ajo 100 o 150 gramos de miga de pan media cebolla un huevo un vaso de vino blanco perejil harina medio vaso de leche aceite de oliva virgen extra y sal empezamos empapando la miga de pan en leche para que vaya ablandando se mientras tanto cortamos la merluza en trocitos muy pequeñitos si no tenéis paciencia para hacerlo con un cuchillo podéis utilizar una picadora eléctrica una vez terminemos de trocear lo metemos en un bol donde después mezclaremos todo

MoviePy - Done.
4
hola
recetasvideos/../temp/receta1_extracto1.wav
El extracto 1 es música o ruido
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
El extracto 3 es un silencio
recetasvideos/../temp/receta1_extracto4.wav
Titulo:Risotto de setas fácil con arroz integral | Cenas vegetarianas saludables
Autor:Avena y Aguacate
Fecha Publicacion:2020-01-06 00:00:00
Entradilla:hola a todos y bienvenidos avena y aguacate este mes os voy a traer todas las semanas recetas vegetarianas riquísimas saludables económicas y muy fáciles de preparar perfectas para el lunes sin carné hoy os traigo mi receta de risotto de setas con arroz integral cremoso buenísimo y encantará a toda la familia para esta receta vamos a usar arroz redondo integral que he dejado toda la noche en remojo para reducir el tiempo de cocción chorizos tradicionales utilizan arroz arborio o carnaroli pero yo prefiero usar arroz integral porque conserva toda su piedra y todos sus nutrientes em

MoviePy - Done.
2
hola
recetasvideos/../temp/receta1_extracto1.wav
El extracto 1 es un silencio
recetasvideos/../temp/receta1_extracto2.wav
Titulo:Leche de Avena Preparacion Casera / Leches Vegetales
Autor:Tanto Monta En La Cocina
Fecha Publicacion:2021-08-24 00:00:00
Entradilla:importante que llevar una dieta balanceada y sobre todo grito por desgracia por lo que estamos pasando por tal motivo el día de hoy quiero compartir contigo cómo hacer leche de avena en casa estoy que estoy más tiempo por lo menos hemos visto que no supermercado es bastante cara y la puedes hacer la comunidad de tu casa y te sabe riquísima y muy económica entre algunos de los beneficios de la leche de avena se sabe qué es buena para el corazón colesterol tiene muchísima fibra tiene mucha proteína tiene mucha vitamina de hierro de un sinfín de beneficios pero eso sí es un poquito alta contrato nosotros ni pretendo serlo simplemente quiero compartir contigo un poquito de lo que hemos aprendido en el jefe qué tal 

MoviePy - Done.
5
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
recetasvideos/../temp/receta1_extracto5.wav
El extracto 5 es un silencio
Titulo:Sopa de Cebolla | Una riquísima receta muy Fácil y Rápida de hacer
Autor:Cocina con Carmen
Fecha Publicacion:2019-12-20 00:00:00
Entradilla:hola a todos y bienvenidos a cocina con Carmen hoy vamos a hacer una deliciosa sopa de cebolla que es súper fácil y rápida de hacer y además es genial para que el cuerpo entre en calor así que si queréis hacer la os dejo los ingredientes abajo en la descripción y al final del vídeo y vamos a comenzar para hacer la sopa de cebolla usaré como ingrediente principal un litro de este caldo de pollo Aneto que está riquísimo yo fui hace poco a su fábrica y la única diferencia que hay con los caldos que hacemos en casa son las ollas que nosotros utilizamos unas ollas pequeñas y ello

MoviePy - Done.
6
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
recetasvideos/../temp/receta1_extracto5.wav
recetasvideos/../temp/receta1_extracto6.wav
El extracto 6 es un silencio
Titulo:¡Pocas personas conocen esta receta! El secreto para un pollo frito crujiente que a todos encantará
Autor:¡Que Viva La Cocina!
Fecha Publicacion:2022-01-25 00:00:00
Entradilla:tenemos unos muslos de pollo a los cuales les hemos quitado la piel de momento salpimentarlos no es necesario lo que vamos a hacer con ellos va a ser introducirlos en abundante agua hirviendo vamos a dejar que cuezan aquí a razón de unos 30 minutos más o menos hasta que los muslos estén bien bien cocinados y lo suficientemente tiernos como para que después podamos desgarrar la carne para hacer esta receta luego lo vemos con un poco más de detalle podrías utilizar también unos muslos de pollo que 

MoviePy - Done.
6
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
recetasvideos/../temp/receta1_extracto5.wav
recetasvideos/../temp/receta1_extracto6.wav
El extracto 6 es un silencio
Titulo:tortilla de patatas. (4 pasos)
Autor:El Rancho de Javier GB
Fecha Publicacion:2013-03-12 00:00:00
Entradilla:la buena hoy voy a preparar una tortilla de patatas para dos personas los ingredientes eran dos patatas grandes una pequeña y dos cebollas y 4 huevos aceite y sal vale pues ya tenemos las patatas cortadas yo las he cortado a taquito y la cebolla cortada los huevos ya los tengo batidos le he echado 4 pizca de sal 1 x huevo y ahora esperaremos que esté el aceite caliente para echar las patatas una vez que está el aceite ya y en caliente echamos primero las patatas movemos y añadimos las cebollas un poquito y para que se haga un pelín más rápido las taparemos han p

MoviePy - Done.
9
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
recetasvideos/../temp/receta1_extracto5.wav
recetasvideos/../temp/receta1_extracto6.wav
recetasvideos/../temp/receta1_extracto7.wav
recetasvideos/../temp/receta1_extracto8.wav
recetasvideos/../temp/receta1_extracto9.wav
El extracto 9 es un silencio
Titulo:Coliflor con Bechamel y Bacon
Autor:El Rancho de Javier GB
Fecha Publicacion:2013-05-07 00:00:00
Entradilla:hola buenas os voy a preparar una coliflor con bechamel de queso y bacon los ingredientes eran una coliflor que ya la tengo cortada cinta te bacon una cebolla y media queso azul leche queso elemental harina aceite y un poco de pimienta nuez moscada y sal pues cuando veamos que el agua comienza el DIF lo primero que haremos será echar la coliflor que el agua ya tiene sal y la dejaremos hervir hasta que se ablande un poco y para que se

MoviePy - Done.
6
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
recetasvideos/../temp/receta1_extracto5.wav
El extracto 5 es un silencio
recetasvideos/../temp/receta1_extracto6.wav
Titulo:Comida fácil.  Pollo al Curry con leche de coco.  Arroz con Pasas
Autor:Smak con Gusto
Fecha Publicacion:2021-02-06 00:00:00
Entradilla:hola mi mi nombre es Pablo nuestro canal se llama Macu un gusto y estamos aquí para hacer verdaderas cositas ricas hoy vamos a preparar un pollo al curry que vos va a flipar bien está receta de pollo al curry pues va a sorprender porque no es un pollo curry tradicional vamos a hacer el pollo al curry indonesio para eso lo que nos va a hacer falta poquitos ingredientes la verdad es que no sale económico y muy muy muy rico si queréis saber cómo darle like a nuestro canal suscribirse abajo y no pierdas ni un minuto vamos a ellos usaré con

In [ ]:
segcionarXsilencios("receta1")

In [84]:
audio1=aseg.from_wav("./recetasvideos/receta1.wav")

In [87]:
type(audio1.duration_seconds)

float

In [112]:
nombre="receta1"
i=4
audio2=aseg.from_wav("temp/{}_extracto{}.wav".format(nombre,i+1))

In [130]:
dep.extraccion('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')


MoviePy - Writing audio in recetasvideos/receta1.wav


MoviePy - Done.
4
hola
recetasvideos/../temp/receta1_extracto1.wav
recetasvideos/../temp/receta1_extracto2.wav
recetasvideos/../temp/receta1_extracto3.wav
recetasvideos/../temp/receta1_extracto4.wav
El extracto 4 es un silencio
Titulo:Albóndigas de Merluza en Salsa Verde | Recetas de PESCADO Faciles
Autor:Platos Fáciles con Tamara
Fecha Publicacion:2013-07-04 00:00:00
Entradilla:hoy vamos a hacer albóndigas de merluza en salsa verde y los ingredientes que vamos a utilizar son 500 gramos de merluza limpios de piel y espinas 3 dientes de ajo 100 o 150 gramos de miga de pan media cebolla un huevo un vaso de vino blanco perejil harina medio vaso de leche aceite de oliva virgen extra y sal empezamos empapando la miga de pan en leche para que vaya ablandando se mientras tanto cortamos la merluza en trocitos muy pequeñitos si no tenéis paciencia para hacerlo con un cuchillo podéis utilizar una picadora eléctrica una vez terminemos de trocear lo metemos en un bol donde después mezclaremos todo